
This notebook will contain a lot of comments that hopefully helps you understand things better. 

**Tip:** I cannot stress this enough, if you are not sure how any function/class/method/concept works there is usually a brief example demonstrating a use case. Then you can copy it and mess around with it. For example use `dir(<object>)` to print its attributes and methods and `type(<object>)` for its type and `print(<object>)` to print its value/contents. This is, in fact, how you learn libraries; by directly interacting with them to see what you can use and how. While you do that you will inevitably encounter errors. **Read what the error says** they are usually informative for deducing where the code went wrong. If you can't figure it out, search for that error online. There is likely many other people that have gone the same path and made the same mistake. 

## Example code to get you started

Let's first create some data. We will use the random package of numpy which let's us generate pseudo-random data from many different types of probability distributions. In this case we will be using normal (Gaussian) distribution in multiple dimensions (multivariate normal). The `mean` parameter denotes the mean of the distribution and `cov` the covariance matrix. If `mean` is N dimensional, `cov` must be NxN. Furthermore, cov must be a "symmetric positive semi-definite matrix" (you can read what PSD means on wikipedia). For our purposes it suffices to say that if we take the outer product of a vector with itself we get a symmetric PSD matrix.

We will be generating two distributions with different means. Dataset A will have a mean \[1,1,1\] and covariance $X_1^TX_1$ and Dataset B will have a mean \[2,1,3\] and covariance $X_2^TX_2$. There is no particular reason we use `np.random.rand` for $X_2$ I just wanted to show that `numpy.random.rand` is another distribution. It generates an array (of size given in parantheses) from the uniform distribution $\mathcal{U}[0,1]$. For reference, `@` performs a matrix multiplication for 2 dimensional arrays in Python, provided that the dimensions match. And double slash `//` does integer division (like integer divisions in C). So the last two rows generate dataset A and dataset B with the given means, covariances are calculated by outer product $X^TX$ and there will be N/2 many points in each dataset denoted by size. 

Digression: if you are wondering how to write code blocks or bold or italic in Jupyter notebooks, search "Markdown syntax" on Google. If you want to use mathematical notation, look up "LaTeX syntax". There are many good entry level tutorials online for both.


In [ ]:
import numpy as np

N=500

X1=np.array([[0.3,-0.2,0.3],[0.3,0.3,0.2],[0.1,0.4,0.5]])
X2=np.random.rand(3,3)
setA=np.random.multivariate_normal(mean=[1,1,1], cov=X1.T@X1, size=N//2)
setB=np.random.multivariate_normal(mean=[2,1,2], cov=X2.T@X2, size=N//2)

**Watch out**: some functions (like `np.random.rand` above) take the dimensions *as regular arguments* and others like `np.zeros` or `np.ones` take the dimensions *in a tuple*. Ex: `np.zeros((5,5))`. If you're unsure which one it is, consult the docs of that package.

## Indexing:

In [ ]:
print("use array[a:b,c:d] to print elements in the array's rows a through b (b not inclusive) and columns c through d (d not inclusive): \n" + str(setA[4:6,0:2]))
print("If you omit the first index it will take it to be 0. If you omit the last index, it will take it to be -1 (meaning last index): \n" + str(setA[:6,1:]))
print("If you omit both, it will print all elements in that index: \n" + str(setA[245:,:]))
print("You can also use negative integers to count backwards from last: \n" + str(setA[-5:,:]))

In [ ]:
# default ending for print is \n. You can change it:
print("Be careful, if you index with a single number, that dimension is squeezed out: " + str(setA[1:2,:]), end="")
print(" vs " + str(setA[1,:]))
# f-string example:
print(f"{setA[:,1:2].shape} and {setA[:,1].shape} are not the same shapes even though they contain the same elements.\nThis might be a problem for example when you are trying to concatenate arrays.")


In [ ]:
import matplotlib.pyplot as plt

# if you are running jupyter notebook in the browser
%matplotlib notebook

# if you are running jupyter notebook in vscode
# %matplotlib widget

# this creates a new figure with the given figsize
# .gca stands for get current axes. without calling that with
# projection='3d' matplotlib assumes the plot is 2d
# don't worry too much about this
fig=plt.figure(figsize=(5,5)).gca(projection='3d')

# scatter takes in x,y,z values. Obtained by indexing different columns
fig.scatter(setA[:,0],setA[:,1],setA[:,2]);
fig.scatter(setB[:,0],setB[:,1],setB[:,2]);



For example here is a 2d projection of the same data on the first two axes:

In [ ]:
# without calling the gca(projection='3d') the default is 2d
plt.figure(figsize=(5,5))
# you can set facecolors, edgecolors, thickness etc
# matplotlib is very flexible
plt.scatter(setA[:,0],setA[:,1], edgecolor =(0.,0.4,0.4), facecolor="xkcd:turquoise");
plt.scatter(setB[:,0],setB[:,1], edgecolor =(0.5,0.,0.7,0.3), facecolor=(0.5,0.,0.5,0.4));

We are concatenating all zeros to `setA` and all ones to `setB` to denote their labels.

In [ ]:
# see how concatenate takes the arrays to be concatenated in a tuple
# also see the dimensions of np.zeros are in a tuple
# that's a lot of parantheses!
setA=np.concatenate((setA,np.zeros((N//2,1))), axis=1)
setB=np.concatenate((setB,np.ones((N//2,1))), axis=1)
print(setB[:10])

And then concatenate setA and setB to create a mixed dataset and shuffle it

In [ ]:
total_dataset=np.concatenate((setA,setB),axis=0)
np.random.shuffle(total_dataset)
print(total_dataset)

## Scikit-learn

scikit-learn's `train_test_split` function gives a convenient way of splitting the data (we could have just as well done this by indexing). Pay attention: we will use test_dataset only once, after we fine tune the hyperparameters.


In [ ]:
from sklearn.model_selection import train_test_split
train_validation_dataset,test_dataset=train_test_split(total_dataset,test_size=0.2)
length=len(train_validation_dataset)

# int() how you type cast in python
train_dataset=train_validation_dataset[:int(length*7/8)]
validation_dataset=train_validation_dataset[int(length*7/8):]


X_train_validation,y_train_validation=train_validation_dataset[:,:-1],train_validation_dataset[:,-1]
X_train,y_train=train_dataset[:,:-1],train_dataset[:,-1]
X_validation,y_validation=validation_dataset[:,:-1],validation_dataset[:,-1]
X_test,y_test=test_dataset[:,:-1],test_dataset[:,-1]



For our purposes there are a few basic *tasks* in data science: regression, classification, clustering, dimensionality reduction etc. (See [here](https://scikit-learn.org/stable/user_guide.html) for various tasks you can use scikit-learn in). In this particular case, we are aiming to trying to tell these two datasets apart i.e. classify them. One of the ways to classify data is using decision trees. To achieve this, we are going to use `DecisionTreeClassifier` *estimator* from `scikit-learn` package.

There are a few basic methods of various estimators that you should pay special attention to:

`fit()`: fits i.e. trains the estimator (also called model sometimes) on the train data and train labels (if labels are available and required by the estimator) Ex: `DecisionTreeClassifier.fit()`, `LinearRegression.fit()`, `KNeighborsClassifier.fit()`

`predict()`: gives its predictions for validation/test data (does not take validation/test labels!) Ex: `KNeighborsClassifier.predict()`, `SVC.predict()`, `SGDRegressor.predict()`

`fit_predict()`: some of the estimators both fit and predict at the same time (the ones that don't require labels such as clustering estimators) Ex: `SpectralClustering.fit_predict()`, `AgglomerativeClustering.fit_predict()`

`transform()`: some of the estimators don't predict but rather *transform* the data, such as dimensionality reduction, projection or preprocessing estimators. Ex: `GaussianRandomProjection.transform()`, `SparseRandomProjection.transform()`, `PCA.transform()`

`fit_transform()`: for the estimators that `transform`, `fit_transform` combines `fit` and `transform` into one operation. Ex: `GaussianRandomProjection.fit_transform()`, `SparseRandomProjection.fit_transform()`, `PCA.fit_transform()`

If you're confused about how a random projection or PCA can `fit` the data and then `transform` it see [this link](https://stackoverflow.com/questions/23838056/what-is-the-difference-between-transform-and-fit-transform-in-sklearn)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt=DecisionTreeClassifier(criterion='entropy', max_depth=4)

# fit() i.e. train the decision tree classifier on the train data
dt.fit(X_train,y_train)

# predict() i.e. get the predictions from the trained classifier on validation data
y_validation_pred=dt.predict(X_validation)

print(f"Validation accuracy: {accuracy_score(y_validation,y_validation_pred)}")

Let's try the same thing with different hyperparameters:

In [ ]:
dt=dt.set_params(criterion='gini', max_depth=10)

# fit() i.e. train the decision tree classifier on the train data
dt.fit(X_train,y_train)

# predict() i.e. get the predictions from the trained classifier on validation data
y_validation_pred=dt.predict(X_validation)

print(f"Validation accuracy: {accuracy_score(y_validation,y_validation_pred)}")

Hmm.. it seems that these are better hyperparameters. Let's go with the second set of hyperparameters. Now let's use the entire `train_validation_dataset` to train the classifier and test it on `test_dataset`.

In [ ]:
# this line is redundant. Since the last set of hyperparameters 
# applied were these, they haven't changed.
dt=dt.set_params(criterion='gini', max_depth=10)

# use all train+validation
dt.fit(X_train_validation,y_train_validation)

y_test_pred=dt.predict(X_test)

print(f"Test accuracy: {accuracy_score(y_test,y_test_pred)}")

If we wanted to see how each individual datapoint is classifier:


In [ ]:
# zip() takes in two arrays and goes through them simultaneously
# it is used instead of 
# for i in range(10):
#    array1[i] ... do something
#    array2[i] ... do something
for predict,label in zip(y_test_pred, y_test):
    print("Predicted class: " +str(predict) + f", true class: {label}")

## Cross Validation and Grid Search

Now in the above example we have not used cross validation (CV) nor did we test many different hyperparameter combinations. To discover the best set of hyperparameters, we can use grid search. This basically means training your model for different combinations of hyperparameters (going over them systematically) and choosing the best one. Luckily for us, there is a function in `scikit-learn` that combines both CV and grid search in one entity. It is called `GridSearchCV`. What this does is it wraps around your estimator and creates another estimator object. While creating this new estimator you specify how many folds you want in your cross validation as well as the parameters to search over (in terms of a Python dictionary). Then, when you call `fit` on this newly created wrapper, it will take care of the grid search and cross validation internally. 

**Tip**: To see an estimator's hyperparameters, you can go to its docs on `scikit-learn` website.

Let's do an example:

In [ ]:
from sklearn.model_selection import GridSearchCV
from time import time

# I want to know how many seconds grid search takes
# time() returns a timestamp in terms of seconds
start=time()

# it doesn't matter if we give initial hyperparameters to DecisionTreeClassifier
# they will be overriden by GridSearchCV anyway
dt=DecisionTreeClassifier()

# see how we specify cv=10 and param_grid as a Python dictionary
# GridSearchCV will go through all combinations of these hyperparameters and find the best one for us
# Make sure the values of the dictionary are Python tuples, lists or numpy arrays
wrapper=GridSearchCV(estimator=dt, cv=10, param_grid={"criterion":('gini','entropy'), "max_depth":np.linspace(1,10,10)})

# we use the whole train_validation_dataset because GridSearchCV does the validation split internally
wrapper.fit(X_train_validation,y_train_validation)

stop=time()
print("Time elapsed: " + str(stop-start))

We can see all results pertaining to the CV and grid search in `.cv_results_` attribute. This includes the 10 splits' accuracies for all hyperparameter combinations (see `split0_test_score`, `split1_test_score` etc) and mean scores (averaged over different splits) (see `mean_test_score`), as well as the list of all parameter combinations (see `params`)

In [ ]:
wrapper.cv_results_

We can get the best score (accuracy) and the hyperparameter combination that achieves the best score like this:

In [ ]:
print("Best hyperparameters are:", wrapper.best_params_)
print("Best accuracy is:", wrapper.best_score_)

We can even directly get the best estimator:

In [ ]:
dt=wrapper.best_estimator_

# an alternative could be
# dt.set_params(**wrapper.best_params_)
# or 
# dt=DecisionTreeClassifier(**wrapper.best_params_)

# FYI: double asterisk ** is used to expand dictionaries as if they are being used as separate arguments.
# For example: 
# dictionary={"a":1, "b":2}
# myfunc(**dictionary) is equivalent to myfunc(a=1, b=2)

Fit the decision tree with the best hyperparameters using `train_validation_dataset` and test on `test_dataset`

In [ ]:
dt.fit(X_train_validation, y_train_validation)
y_test_pred=dt.predict(X_test)
print("Accuracy:",accuracy_score(y_test,y_test_pred))

An alternative to `GridSearchCV` is to use `StratifiedKFold` and `ParameterGrid`. These are used to implement K-fold CV and grid search respectively. `StratifiedKFold` gives us the indices of train and validation splits so we can use them in a for loop. `ParameterGrid` on the other hand generates the Cartesian product of all the parameter lists it is given. See the example below:

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid

# create a StratifiedKfold object
skf = StratifiedKFold(n_splits=10)

param_options = {"criterion":('gini','entropy'), "max_depth":np.linspace(1,10,10)}
param_grid=ParameterGrid(param_options) # this will give an iterator that iteratos over all possible hyperparameter combinations
print(param_grid[0])
print(param_grid[1])

Then we can use two nested for loops: one that cycles through the hyperparameter combinations, the other cycles through different validation/train data splits.

In [ ]:

# create an array of zeros to keep mean_accuracies (averaged over splits) for each hyperparameter combination
mean_accuracies=np.zeros(len(param_grid))

# measure time
start=time()

for i,param_dict in enumerate(param_grid):
    # create an empty array to keep accuracies for different CV folds
    fold_accuracies=[]

    # set the hyperparameters to current iteration's combination
    dt.set_params(**param_dict)

    for train_index, validation_index in skf.split(X_train_validation,y_train_validation):
        # skf gave us train and validation indices. use them
        X_train, X_validation = X_train_validation[train_index], X_train_validation[validation_index]
        y_train, y_validation = y_train_validation[train_index], y_train_validation[validation_index]

        # train the model with these hyperparameters and data
        dt.fit(X_train, y_train)
        y_predict=dt.predict(X_validation)

        # append current fold's accuracy to fold_accuracies
        fold_accuracies.append(accuracy_score(y_validation, y_predict))

    # record the accuracy averaged across folds to mean_accuracies 
    mean_accuracies[i]=np.mean(fold_accuracies)
    
stop=time()

print("Time: "+ str(stop-start))
print("Mean accuracies: "+str(mean_accuracies))


In [ ]:
max_index=np.argmax(mean_accuracies)
print("Max accuracy: "+str(mean_accuracies[max_index]))
optimal_hyperparameters=param_grid[max_index]
print("Hyperparameters achieving max accuracy: " +str(optimal_hyperparameters))

In [ ]:
dt.set_params(**optimal_hyperparameters)
dt.fit(X_train,y_train)
y_test_predict=dt.predict(X_test)
print(accuracy_score(y_test, y_test_predict))